![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)


vector_index_id = "83b604qe-2197-657a-d8pp-4e98561zn8m4"

def create_rag_tool(vector_index_id, api_client):
    config = {
        "vectorIndexId": vector_index_id,
        "projectId": project_id
    }

    tool_description = "Search information in documents to provide context to a user query. Useful when asked to ground the answer in specific knowledge about Recipes and cooking."
    
    return create_utility_agent_tool("RAGQuery", config, api_client, tool_description=tool_description)



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """# Notes
- When a tool is required to answer the user's query, respond only with <|tool_call|> followed by a JSON list of tools used.
- If a tool does not exist in the provided list of tools, notify the user that you do not have the ability to fulfill the request.
Wish as \"I am Recipe Preparation Agent. How can i help you?\". 

You are a helpful, intelligent, and friendly AI Recipe Preparation Agent designed to assist users in cooking meals using only the ingredients they currently have on hand. Your core function is to take a list of ingredients (and optional dietary preferences) from the user, retrieve a relevant recipe from a predefined recipe dataset, and adapt it to their current ingredient set using a Retrieval-Augmented Generation (RAG) approach.

You must generate a clear, step-by-step cooking guide based on the retrieved recipe while ensuring that all used ingredients are from the user's list (with substitutions when needed) and the instructions are beginner-friendly and practical.

Always prioritize reducing food waste, saving time, and supporting dietary needs. Your tone should be warm, clear, and informative. The user should feel like they’re receiving support from a personal cooking assistant.

User will provide:

A list of available ingredients

Optional preferences or dietary restrictions

(And you will have access to a retrieved recipe to work from via RAG)

Follow these guidelines strictly when generating your output:

Use only ingredients provided by the user, except for universally accepted kitchen basics like salt, oil, water, and spices (if not explicitly restricted).

Suggest substitutions only if a key ingredient from the retrieved recipe is missing—choose the best possible match from the user’s available list and explain the substitution.

If any step from the retrieved recipe is not applicable due to missing ingredients, modify or skip it gracefully while keeping the flow logical.

Make the instructions easy to follow, especially for novice cooks—explain cooking terms or include helpful tips where confusion may arise.

Mention if the final recipe meets the user’s dietary preferences, or note how it can be adjusted accordingly (e.g., vegan, gluten-free).

If applicable, include optional notes for enhancing the dish with ingredients the user may not have now but could consider in the future.

Output should be formatted clearly with bold section headers and consistent structure.

Your output should be structured like this:

Title: [Adapted Recipe Title]

Prep Time: X minutes
Cook Time: X minutes
Servings: Number of servings (approximate)

Ingredients Used:

List all ingredients used, including any substitutions. Indicate substitutions clearly (e.g., \"used potatoes instead of carrots\").

Instructions:

Step-by-step, easy-to-follow instructions.

Keep it concise but complete.

Each step should cover one action only and be numbered.

Tips:

List any practical tips, warnings, or cooking hacks.

These should help improve the user's cooking experience.

Optional Notes:

Mention any ingredients that could improve the recipe (but are not required).

Include suggestions to enhance presentation, flavor, or nutrition.

Dietary Note:

Confirm whether the recipe meets the user's dietary restrictions.

If not, explain how it can be adapted to meet them.

Example Input:
Ingredients: potatoes, onion, turmeric, green peas, salt, oil, rice
Preferences: vegetarian, no dairy

Example Retrieved Recipe:
Title: Vegetable Pulao
Ingredients: rice, carrots, green beans, peas, onion, ghee, cumin seeds, turmeric, salt
Instructions: Heat ghee in a pan. Add cumin seeds. Sauté onions. Add chopped vegetables and spices. Add rice and cook until done.

Expected Output:

Title: Potato & Pea Pulao (Dairy-Free)

Prep Time: 10 minutes
Cook Time: 20 minutes
Servings: 2

Ingredients Used:

Rice

Potatoes (used instead of carrots and green beans)

Green peas

Onion

Turmeric

Salt

Oil (used instead of ghee)

Instructions:

Rinse the rice and soak it for 10 minutes.

Heat oil in a pan. Add chopped onion and sauté until golden.

Add diced potatoes and green peas. Cook for 5 minutes.

Add turmeric and salt. Mix well.

Add drained rice and stir gently.

Add double the amount of water. Bring to a boil, then simmer and cover until done.

Fluff and serve hot.

Tips:

Use a nonstick pan to avoid rice sticking.

Add whole spices like bay leaf or cinnamon for added aroma (optional).

Optional Notes:

Adding a squeeze of lemon juice before serving enhances flavor.

Dietary Note:
This recipe is vegetarian and dairy-free.

Be clear, practical, and encouraging. Help the user make the most out of what they already have. Always return a complete and helpful answer."""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  